In [1]:
!pip install lightning

In [2]:
!rm -rf /kaggle/working/*

In [3]:
"""
Imports
"""

import os
from dataclasses import dataclass

import lightning as L
import lightning.pytorch as pl
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as T
import torchvision.transforms.functional as TF
from lightning.pytorch.callbacks import EarlyStopping, StochasticWeightAveraging, LearningRateMonitor, ModelCheckpoint, TQDMProgressBar
from lightning.pytorch.loggers import TensorBoardLogger
from sklearn.model_selection import train_test_split

torch.set_float32_matmul_precision('medium')

In [4]:
@dataclass
class TrainingConfig:
    image_size = 28  # the generated image resolution

    train_batch_size = 256
    val_batch_size = 256

    max_epochs = 100
    check_val_every_n_epoch = 10
    log_every_n_steps = 50
    accumulate_grad_batches = 1
    learning_rate = 1e-6

    output_dir = "/kaggle/working"

    seed = 10


config = TrainingConfig()

In [5]:
transform = T.Compose([
    T.Normalize([0.5], [0.5]),
])

reverse_transform = T.Compose([
    T.ToTensor(),
    T.Normalize([-0.5/0.5], [1/0.5]),
    T.ToPILImage(),
])


In [6]:
class MNISTDataset(torch.utils.data.Dataset):
    def __init__(self, labels, images, transform=None, pred=False):
        super().__init__()
        self.labels = labels
        self.images = images
        self.pred = pred
#         assert len(self.labels) == len(self.images)

        self.transform = transform

    def __len__(self):
        length = len(self.images)
        return length

    def __getitem__(self, index):
        image = self.images[index]
        image = image.unsqueeze(0)

        if self.transform:
            image = self.transform(image)
            
        if not self.pred:
            label = self.labels[index]
            return label, image

        return image

In [7]:
class MNISTDataModule(L.LightningDataModule):
    def __init__(self,
                 train_transform,
                 test_transform):
        super().__init__()
        self.num_workers = os.cpu_count()  # <- use all available CPU cores

        self.train_transform = train_transform
        self.test_transform = test_transform

        self.train_dataset = None
        self.val_dataset = None
        self.predict_dataset = None

    def setup(self, stage: str):
        if stage == "fit":
            all_data = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")

            train_data = all_data.iloc[5000:, :]
            val_data = all_data.iloc[:5000, :]

            train_labels = torch.tensor(train_data.label.values)
            val_labels = torch.tensor(val_data.label.values)

            # Reshaping data
            train_images = train_data.iloc[:, 1:].values.reshape(37000, 28, 28)
            val_images = val_data.iloc[:, 1:].values.reshape(5000, 28, 28)

            train_images = torch.from_numpy(train_images).type(torch.float32)
            val_images = torch.from_numpy(val_images).type(torch.float32)

            # train_images = train_images / 255
            # val_images = val_images / 255

            self.train_dataset = MNISTDataset(
                labels=train_labels,
                images=train_images,
                transform=self.train_transform
            )

            self.val_dataset = MNISTDataset(
                labels=val_labels,
                images=val_images,
                transform=self.test_transform
            )

            print(f"Total Dataset       : {len(self.train_dataset) + len(self.val_dataset)} samples")
            print(f"Train Dataset       : {len(self.train_dataset)} samples")
            print(f"Validation Dataset  : {len(self.val_dataset)} samples")

        if stage == 'predict':
            samples = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")
            images = samples.iloc[:, :].values.reshape(28000, 28, 28)
            images = torch.from_numpy(images).type(torch.float32)
            # images = images / 255

            self.predict_dataset = MNISTDataset(
                labels=None,
                images=images,
                transform=self.test_transform,
                pred=True
            )

            print(f"Predict Dataset  : {len(self.predict_dataset)} samples")

    def train_dataloader(self):
        return torch.utils.data.DataLoader(
            self.train_dataset,
            batch_size=config.train_batch_size,
            shuffle=True,
            num_workers=self.num_workers,
            persistent_workers=True,
            pin_memory=True,
        )

    def val_dataloader(self):
        return torch.utils.data.DataLoader(
            self.val_dataset,
            batch_size=config.val_batch_size,
            shuffle=False,
            num_workers=self.num_workers,
            persistent_workers=True,
            pin_memory=True,
        )
    
    def predict_dataloader(self):
        return torch.utils.data.DataLoader(
            self.predict_dataset,
            batch_size=config.val_batch_size,
            shuffle=False,
            num_workers=self.num_workers,
            persistent_workers=True,
            pin_memory=True,
        )

In [8]:
"""
Model
"""


class MNISTModel(nn.Module):
    def __init__(self):
        super().__init__()

        self.block_1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=512, kernel_size=5, stride=2),
            nn.ELU(),
            nn.Conv2d(in_channels=512, out_channels=256, kernel_size=5),
            nn.ELU(),
            nn.Conv2d(in_channels=256, out_channels=128, kernel_size=5),
            nn.ELU(),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=2),
            nn.ELU()
        )

        self.block_2 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=64, kernel_size=3),
            nn.ELU(),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=1),
            nn.ELU(),
            nn.Conv2d(in_channels=64, out_channels=32, kernel_size=1),
            nn.ELU(),
            nn.Conv2d(in_channels=32, out_channels=10, kernel_size=1),
        )

        self.classifier = nn.Sequential(
            nn.LogSoftmax(dim=1),
        )

    def forward(self, x):
        x = self.block_1(x)
#         print(f"shape : {x.shape}")
        x = self.block_2(x)
        x = self.classifier(x)
        x = torch.squeeze(x)
        return x


In [9]:
"""
Lightning Module
"""


class LightningMNIST(pl.LightningModule):
    def __init__(self,
                 model):
        super().__init__()

        self.model = model

        total_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
        print(f"Number of Trainable Parameters : {total_trainable_params}")

        self.learning_rate = config.learning_rate

        self.save_hyperparameters(ignore=['model'])

    def forward(self, images):
        labels_logits = self.model(images)
        return labels_logits

    def shared_step(self, batch):
        labels, images = batch
        labels_logits = self.model(images)

        loss = F.nll_loss(labels_logits, labels)
        return loss

    def training_step(self, batch, batch_idx):
        loss = self.shared_step(batch)
        self.log("train_loss", loss, prog_bar=True)
        return loss

    @torch.no_grad()
    def validation_step(self, batch, batch_idx):
        loss = self.shared_step(batch)
        self.log("val_loss", loss, prog_bar=True)
        return loss

    @torch.no_grad()
    def test_step(self, batch, batch_idx):
        loss = self.shared_step(batch)
        self.log("test_loss", loss, prog_bar=True)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.model.parameters(), lr=self.learning_rate)
        lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)
        # lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=2)

        return {
            "optimizer": optimizer,
            # "monitor": "train_loss",
            "lr_scheduler": {
                "scheduler": lr_scheduler,
                "monitor": "train_loss",
            }
        }

    def configure_callbacks(self):
        early_stop = EarlyStopping(
            monitor="val_loss",
            mode="min",
            min_delta=0.00,
            patience=3,
            verbose=False,
        )

        checkpoint = ModelCheckpoint(
            monitor='val_loss',
            mode='min',
            dirpath=f'{config.output_dir}/checkpoints/',
            save_top_k=1,
            save_last=True
        )

        progress_bar = TQDMProgressBar(process_position=0)
        lr_monitor = LearningRateMonitor(logging_interval='step')
        # summary = ModelSummary(max_depth=-1)
        # swa = StochasticWeightAveraging(swa_lrs=1e-2)

        return [checkpoint, progress_bar, lr_monitor, early_stop]

In [10]:

"""
Main Function
"""

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

model = MNISTModel()

lightning_module = LightningMNIST(
    model
)

dm = MNISTDataModule(
    train_transform=transform,
    test_transform=transform
)

trainer = pl.Trainer(
    default_root_dir=f"{config.output_dir}/",
    logger=TensorBoardLogger(save_dir=f'{config.output_dir}/'),
    devices='auto',
    accelerator="auto",  # auto, gpu, cpu, ...

    max_epochs=config.max_epochs,
    log_every_n_steps=config.log_every_n_steps,
    check_val_every_n_epoch=config.check_val_every_n_epoch,
    accumulate_grad_batches=config.accumulate_grad_batches,
    # gradient_clip_val=0.1,

    # fast_dev_run=True,
    # overfit_batches=1,
    num_sanity_val_steps=1,
    enable_model_summary=False,
)

trainer.fit(
    lightning_module,
    datamodule=dm,
#     ckpt_path=f'{config.output_dir}\\checkpoints\\last.ckpt'
)

best_model_path = trainer.checkpoint_callback.best_model_path
print(f"Best model path : {best_model_path}")

Using device: cuda
Number of Trainable Parameters : 4255722


INFO: GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO: The following callbacks returned in `LightningModule.configure_callbacks` will override existing callbacks passed to Trainer: ModelCheckpoint, TQDMProgressBar
2024-05-02 08:28:27.487387: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-02 08:28:27.487438: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-02 08:28:27.488931: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
INFO: LOCAL_RANK: 0 - CU

Total Dataset       : 42000 samples
Train Dataset       : 37000 samples
Validation Dataset  : 5000 samples


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=100` reached.


Best model path : /kaggle/working/checkpoints/epoch=99-step=14500.ckpt


In [21]:
!ls

checkpoints  directory.zip  lightning_logs  mnist_submission.csv


In [22]:
import zipfile
import os
from IPython.display import FileLink

def zip_dir(directory = os.curdir, file_name = 'directory.zip'):
    """
    zip all the files in a directory
    
    Parameters
    _____
    directory: str
        directory needs to be zipped, defualt is current working directory
        
    file_name: str
        the name of the zipped file (including .zip), default is 'directory.zip'
        
    Returns
    _____
    Creates a hyperlink, which can be used to download the zip file)
    """
    os.chdir(directory)
    zip_ref = zipfile.ZipFile(file_name, mode='w')
    for folder, _, files in os.walk(directory):
        for file in files:
            if file_name in file:
                pass
            else:
                zip_ref.write(os.path.join(folder, file))

    return FileLink(file_name)

In [23]:
zip_dir()

/kaggle/working/directory.zip

In [17]:
labels_logits = trainer.predict(
    ckpt_path=best_model_path,
    dataloaders=dm
)

INFO: The following callbacks returned in `LightningModule.configure_callbacks` will override existing callbacks passed to Trainer: EarlyStopping, LearningRateMonitor, ModelCheckpoint, TQDMProgressBar
INFO: Restoring states from the checkpoint path at /kaggle/working/checkpoints/epoch=99-step=14500.ckpt
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: Loaded model weights from the checkpoint at /kaggle/working/checkpoints/epoch=99-step=14500.ckpt


Predict Dataset  : 28000 samples


Predicting: |          | 0/? [00:00<?, ?it/s]

In [18]:
pred_labels = np.argmax(torch.cat(labels_logits, dim=0).detach().numpy(), axis=1)
pred_labels.shape

(28000,)

In [16]:
submission = pd.concat([
    pd.Series(range(1,28001), name = "ImageId"), 
    pd.Series(pred_labels, name="Label")], 
    axis = 1
)

submission.to_csv("mnist_submission.csv", index=False)